<a href="https://colab.research.google.com/github/restartus/covid-projection/blob/rich-demo/model/altair/Restart_Altair_Experiments_2020_07_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Altair Tutorial

[Altair tutorial](https://altair-viz.github.io/altair-tutorial/README.html) is a good way to learn things.

In [ ]:
import altair as alt
from vega_datasets import data

In [ ]:
cars = data.cars()

In [ ]:
cars.head()

In [ ]:
alt.Chart(cars).mark_point().encode(x='Miles_per_Gallon')

In [ ]:
alt.Chart(cars).mark_tick().encode(y='Horsepower')

In [ ]:
# Now a 2-D chart just specifies x and y
alt.Chart(cars).mark_point().encode(y='Horsepower', x='Miles_per_Gallon')

In [ ]:
# Panning and zooming with interactive as another modifer in the chain
alt.Chart(cars).mark_point().encode(x='Displacement', y='Horsepower').interactive()

In [ ]:
# Now we can use color as another dimension
alt.Chart(cars).mark_point().encode(x='Miles_per_Gallon', y='Horsepower', color='Origin')

    # Now tell Altair that the values are ordina
    alt.Chart(cars).mark_point().encode(x='Miles_per_Gallon', y='Horsepower', color='Cylinders:=O'

In [ ]:
alt.Chart(cars).mark_point().encode(x='Miles_per_Gallon', y='Displacement', color='Cylinders:O')

# Using some of our data

In [ ]:
import numpy as np
import pandas as pd

class Model():
  label = { "Level" : ["Essential", "Non-essential"],
             "Resource" : ["N95 Medical", "N95", "ASTM 3 Mask", "ASTM 2 Mask", "Mask", "Gloves", "Gowns", "Goggles"],
            }
  dim = { "n": 7,
         "l": 2}
class Resource():
  foo = np.array([[123, 2313, 34543, 4343, 556, 208, 34, 43453],
                 [345, 34234, 2343,8787, 8778, 4545, 3454, 423]])
  def __init__(self, model):
    self.demand_ln_arr = np.array([[500, 14, 1030, 1232, 234, 341, 1234, 343],
                                   [12, 34, 56, 1232, 10343, 234, 9090, 32343]])
    self.demand_ln_df = pd.DataFrame(self.foo,
                            index=model.label["Level"],
                            columns=model.label["Resource"])
    self.demand_ln_df.index.name = "Level"

model = Model()
res = Resource(model)
print(f"{res.demand_ln_df}")
res.demand_ln_arr

# https://altair-viz.github.io/user_guide/data.html
# https://stackoverflow.com/questions/18022845/pandas-index-column-title-or-name
res.demand_ln_df.shape
res.demand_ln_df.ndim
res.demand_ln_df.index
# res.demand_ln_df.index.name="Level"
res.demand_ln_df.rename_axis(index='Level', inplace=True)
print(f'{res.demand_ln_df}')
res.demand_ln_df.shape
res.demand_ln_df.reset_index()
print(f'{res.demand_ln_df.reset_index()}')

In [ ]:
# https://pbpython.com/altair-intro.html
# :Q is quantitiatve, :O is ordinal
alt.Chart(res.demand_ln_df.reset_index()).mark_point().encode(x='N95 Medical:Q', y='Level')

In [ ]:
# Pandas melt to convert from wide from to long form
# https://altair-viz.github.io/user_guide/data.html
d = res.demand_ln_df
d.columns
d.index.name="Level"
d=d.reset_index()

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html
d.melt(id_vars=['Level'])
# https://altair-viz.github.io/user_guide/data.html
dmelt=d.melt('Level', var_name='Resource', value_name='Units')

# alt.Chart(res.demand_ln_df.melt()).mark_bar().encode(x='variable', y='value')

In [ ]:
# https://altair-viz.github.io/getting_started/starting.html
# This takes the level and spreads all the units across it
# in long form data, there are value columns (in this case only one)
# and a bunch of index column in this case there are two
# So this generates a one dimensional
alt.Chart(dmelt).mark_point().encode(x="Level")

In [ ]:
alt.Chart(dmelt).mark_bar().encode(x="Resource", y="Units")

In [ ]:
# https://altair-viz.github.io/getting_started/starting.htm
import pandas as pd
data = pd.DataFrame({'a': list('CCCDDDEEE'),
                     'b': [2, 7, 4, 1, 2, 6, 8, 4, 7]})
data

In [ ]:
alt.Chart(data).mark_point().encode(x='a', y='b')

In [ ]:
# https://altair-viz.github.io/gallery/grouped_bar_chart.html
alt.Chart(dmelt).mark_bar().encode(
    x='Level',
    y='Units',
    color='Level',
    column='Resource'
)

In [ ]:
# Combo hard are easy
# just have a base then encode from there
# with different y values
# https://altair-viz.github.io/gallery/bar_and_line_with_dual_axis.html
chart = alt.Chart(dmelt).encode(x='Level')
# Note how easy it is to just add more properties
column_chart = chart.mark_bar().encode(column='Resource', y='Units', color="Level")
bar = chart.mark_bar().encode(y='Units')
# https://altair-viz.github.io/gallery/bar_chart_with_mean_line.html
# note you cannot do this for things with columns as in column_chart
line = chart.mark_line(color='red').encode(y='mean(Units)')
(bar + line).properties(width=600)


In [ ]:
column_chart = chart.mark_bar().encode(column='Resource', y='Units', color="Level")
column_chart

In [ ]:
# https://altair-viz.github.io/gallery/bar_rounded.html
chart.mark_bar(cornerRadiusTopLeft=3,
              cornerRadiusTopRight=3
).encode(
    x='Level',
    y="Units",
    column='Resource',
    color='Level'
)

In [ ]:
# https://altair-viz.github.io/gallery/percentage_of_total.html
# doing math inside altair 
# https://altair-viz.github.io/user_guide/transform/joinaggregate.html
# We are creating a new column of data, so we perform
# an aggregation and then join it to the original dataset in place
alt.Chart(dmelt).transform_joinaggregate(
    TotalUnits = 'sum(Units)'
    ).transform_calculate(
        PercentOfTotal='datum.Units / datum.TotalUnits'
    ).mark_bar().encode(
        x='PercentOfTotal:Q',
        y='Level',
        column='Resource'  
    )

In [ ]:
# https://altair-viz.github.io/user_guide/transform/calculate.html
# https://github.com/altair-viz/altair/pull/215
# The above is quite messy because you are passing strings
# And are using vega-lite underneath
from altair import expr, datum

data = expr